# Understanding Backpropagation by doing it from scratch

### Importing necessary libraries

In [2]:
import random
import torch
import torch.nn.functional as F
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

### Reading the data

In [3]:
words = open("./data/names.txt", "r").read().splitlines()
len(words)

32033